In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

### Load the experiment and corresponding libraries

In [2]:
project_path = os.path.join(Path.home(), "projects")
sys.path.append(project_path)  # enable import of "bias_transfer_recipes"
os.chdir(os.path.join(project_path,"bias_transfer_recipes/work/"))

In [3]:
from bias_transfer_recipes.bias_transfer_recipes.main import load_experiment
experiment = load_experiment(recipe="_2021_09_24_aistats", 
                             experiment="mnist_st", 
                             schema="",  # schema name will automatically be generated from recipe and experiment names 
                             base_dir=os.path.join(project_path,"bias_transfer_recipes/bias_transfer_recipes/"),
                             import_prefix="bias_transfer_recipes.bias_transfer_recipes.", 
                             dev_mode=False  
                            )

Connecting anix@134.76.19.44:3306


In [4]:
from nntransfer.tables.transfer import *
from nntransfer.tables.nnfabrik import *
from nntransfer.analysis.plot import plot, save_plot
from nntransfer.configs.base import Description
import nnfabrik as nnf
schema

Connecting anix@134.76.19.44:3306


Schema `bias_transfer__2021_09_24_aistats`

### Retrieve orbit experiment and plot kernels + transformed inputs

In [5]:
experiment.transfer_experiments.keys()

dict_keys([Description(name='MNIST Experiment Teacher Translation Invariant', seed=42), Description(name='Translation Invariant Equivariance Transfer (first_layer_transform: False, id:10.0, equiv:1.0, inv:1.0, ce:1.0)', seed=42), Description(name='Translation Invariant Equivariance Transfer (first_layer_transform: False, id:10.0, equiv:1.0, inv:1.0, ce:0.0)', seed=42), Description(name='Translation Invariant Equivariance Transfer (first_layer_transform: False, id:10.0, equiv:0.0, inv:1.0, ce:1.0)', seed=42), Description(name='Translation Invariant Equivariance Transfer (first_layer_transform: False, id:10.0, equiv:0.0, inv:1.0, ce:0.0)', seed=42), Description(name='Translation Invariant Equivariance Transfer (first_layer_transform: False, id:10.0, equiv:1.0, inv:0.0, ce:1.0)', seed=42), Description(name='Translation Invariant Equivariance Transfer (first_layer_transform: False, id:10.0, equiv:1.0, inv:0.0, ce:0.0)', seed=42), Description(name='Translation Invariant Equivariance Transfe

In [6]:
exp = experiment.transfer_experiments[Description(name='Translation Invariant Equivariance Transfer (repititions:1, id_between_filters:True, id_factor:10.0)', seed=42)]
TransferredTrainedModel() & exp.get_restrictions(1)

KeyError: Description(name='Translation Invariant Equivariance Transfer (repititions:1, id_between_filters:True, id_factor:10.0)', seed=42)

In [7]:
import tempfile
import torch
with tempfile.TemporaryDirectory() as temp_dir:
    state = ((TransferredTrainedModel() & exp.get_restrictions(1)) * TransferredTrainedModel.ModelStorage()).fetch1("model_state", download_path=temp_dir)
    state_dict = torch.load(state)
kernels = state_dict["kernels"].cpu().numpy()
print(kernels.shape)

from nntransfer.analysis.plot import plot
@plot
def plot_kernels(kernels, fig, ax):
    for i in range(5):
        for j in range(5):
            ax[i][j].imshow(kernels[i*5+j], cmap="viridis")
#             ax[i][j].set_title(i*5+j)
            ax[i][j].axis('off')
fig, ax = plot_kernels(kernels, nrows=5, ncols=5, tight=False, style="light_nips_tex", save="kernels_translation_clean", fraction=0.6)

KeyboardInterrupt: 

In [5]:
from bias_transfer.dataset.torchvision_loader import torchvision_dataset_loader
conf = (Dataset() & {"dataset_hash": "34cf5fe50a3e4763063dce5971dc7ccb"}).fetch1("dataset_config")
data = torchvision_dataset_loader(42, **conf)
test = data["test"]["img_classification"]
img = next(iter(test))[0][:1]

Loading dataset: MNIST
Images already downloaded in ./data/image_classification/torchvision/MNIST
Images already downloaded in ./data/image_classification/torchvision/MNIST-C


In [6]:
from bias_transfer.models.learned_equiv import LearnedEquivariance


@plot
def plot_images(kernels, img, fig, ax):
    model = LearnedEquivariance(kernel_size=5,group_size=25)
    model.eval()
    model.kernels.data = torch.tensor(kernels)
    for i in range(5):
        for j in range(5):
            filter_out = model(img, g=torch.ones(img.shape[0],dtype=int)*(i*5+j),l=0,n=1).cpu().detach().numpy()
            ax[i][j].imshow(filter_out.reshape(28,28,1), cmap="viridis")
#             ax[i][j].set_title(i*5+j)
            ax[i][j].axis('off')

fig, ax = plot_images(kernels, img, nrows=5, ncols=5, tight=False, style="light_nips_tex", save="images_translation_clean", fraction=0.5)
    

NameError: name 'kernels' is not defined

### Retrieve all results across method and display as table

In [5]:
from bias_transfer.analysis.general_results import *
analyzer = ResultsAnalyzer()

In [6]:
analyzer.load_data(experiment.transfer_experiments)

In [7]:
df = analyzer.generate_table(objectives=(("Test","img_classification","accuracy"),
                                         ("translate","img_classification","accuracy"),
                                         ("Rotation Test","img_classification","accuracy"),
                                         ("Noise noise_std 0.08_0.2.0.12_0.2.0.18_0.2.0.26_0.2.0.38_0.2","img_classification","accuracy")),
                            last_n=1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)
df.sort_values(['name', '0: translate'], ascending=[True, False])
df

,0: Test,0: translate,0: Rotation Test,alpha
name,,,,
MNIST Experiment Student,98.31,32.38,39.48,None
MNIST Experiment Teacher Rotation Invariant,99.14,59.61,87.65,None
MNIST Experiment Teacher Translation Invariant,98.93,90.67,42.33,None
Rotation Invariant Equivariance Transfer G,97.67,30.63,74.24,25
Rotation Invariant Equivariance Transfer G,97.65,14.16,61.68,4
Rotation Invariant Equivariance Transfer G,93.10,24.87,67.35,8
Translation Invariant Equivariance Transfer G,97.40,20.29,42.95,25
Translation Invariant Equivariance Transfer G,97.91,25.56,39.75,4
Translation Invariant Equivariance Transfer G,97.57,19.92,39.50,8


In [10]:
table = ""
for idx, row in df.iterrows():
    alpha = row["alpha"]
    if alpha is None:
        continue
    alpha = alpha[:-1]
    alpha = alpha.split(",")[1:]
    weights = {s.split(":")[0].strip():float(s.split(":")[1]) for s in alpha}
    for weight in weights.values():
        table += "x" if weight > 0 else "-"
        table += " & "
    table += str(row["0: Test"]) + " & "
    table += str(row["0: translate"]) + " \\\\ \n"
print(table)

x & x & x & x & 99.02 & 96.48 \\ 
x & x & x & - & 99.02 & 96.48 \\ 
x & - & x & x & 89.98 & 88.65 \\ 
x & - & x & - & 92.82 & 89.68 \\ 
x & x & - & x & 98.36 & 89.98 \\ 
x & x & - & - & 98.45 & 93.8 \\ 
x & - & - & x & 37.25 & 52.82 \\ 
x & - & - & - & 43.38 & 58.69 \\ 
- & x & x & x & 98.78 & 85.01 \\ 
- & x & x & - & 98.48 & 81.18 \\ 
- & - & x & x & 98.37 & 89.54 \\ 
- & - & x & - & 98.59 & 90.98 \\ 
- & x & - & x & 98.24 & 77.36 \\ 
- & x & - & - & 98.05 & 83.51 \\ 

